<a href="https://colab.research.google.com/github/VrindaM27/Social-Web-Mobile-Analytics-Labwork/blob/main/SWM_Lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd

In [4]:
pip install --upgrade google-api-python-client

In [2]:
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
import httplib2

In [3]:
credentials = ServiceAccountCredentials.from_json_keyfile_name('client_secrets.json', ['https://www.googleapis.com/auth/analytics.readonly'])

In [5]:
#Create a service object
http = credentials.authorize(httplib2.Http())
service = build('analytics', 'v4', http=http, discoveryServiceUrl=('https://analyticsreporting.googleapis.com/$discovery/rest'))

In [12]:
body={'reportRequests': [
            {
                'viewId': '225899521', 
                'dateRanges': [{'startDate': '30daysAgo', 'endDate': 'today'}],
                'metrics': [{'expression': 'ga:sessions'}], 
                'dimensions': [{"name": "ga:pagePath"}], #Get Pages
                "filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate", #Filter by condition "containing products"
                'orderBys': [{"fieldName": "ga:sessions", "sortOrder": "DESCENDING"}], 
                'pageSize': 100
            }]
    }

In [20]:
dim = []
val = []
  
#Extract Data
for report in body.get('reports', []):
  
    columnHeader = report.get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
    rows = report.get('data', {}).get('rows', [])
  
    for row in rows:
  
        dimensions = row.get('dimensions', [])
        dateRangeValues = row.get('metrics', [])
  
        for header, dimension in zip(dimensionHeaders, dimensions):
            dim.append(dimension)
  
        for i, values in enumerate(dateRangeValues):
            for metricHeader, value in zip(metricHeaders, values.get('values')):
                val.append(int(value))

In [23]:
df = pd.DataFrame() 
df["Sessions"]=val
df["pagePath"]=dim
df=df[["pagePath","Sessions"]]

In [27]:
df.to_csv("page_by_session.csv")